## This notebook cleans and prepares the data tables provided from the startup:

1) user account creation data <br>
2) user answers to the pre-assessment questions <br>
3) answer key for pre-assessment questions <br>
4) question key for pre-assessment questions <br>
4) sleep tracker data <br>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA

from ipywidgets import FloatProgress
from IPython.display import display
import time

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read in data

Read data from the three tables: <br>
1) users' unique app data at account creation <br>
2) user answers to the pre-assessment questions <br>
3) answer key for pre-assessment questions (possible answers to all questions) <br>

In [5]:
# user initial app data
users_initial = pd.read_csv("//Users/Maime/Dropbox/Me/Insight/Shleep/user_data/users-anon.csv")
# user's answers to all questions
answers = pd.read_csv("//Users/Maime/Dropbox/Me/Insight/Shleep/user_data/user_question_answers.csv")
# key of all possible answers users can give in pre-assessment
answer_key = pd.read_csv("//Users/Maime/Dropbox/Me/Insight/Shleep/user_data/answers.csv")

# there are 3 versions of the pre-assessment, only use 2nd and 3rd
V2_users = answers[answers['question_id'] == 52]['user_id'].unique()
V3_users = answers[answers['question_id'] == 73]['user_id'].unique()
unique_user_ids = np.append(V2_users, V3_users)
# since V2 and V3 pre-assessments are identical aside from gender, we can recode the question_id to be the same
V2_question_id = list(range(35, 53))
V3_question_id = list(range(55, 74))
V3_question_id.remove(58)

# new dataframe with just users who completed V2 or V3
answers.question_id.replace(V2_question_id, V3_question_id, inplace = True)
answers_complete = answers.loc[answers['user_id'].isin(unique_user_ids)]
answers_complete = answers_complete.sort_values(['user_id', 'question_id'])
# remove rows with question_id 1-54
temp = range(1, 55)
for q in temp:
    answers_complete = answers_complete[answers_complete.question_id != q]
# which question_id's are still left?
temp = answers_complete['question_id'].unique()
np.sort(temp)

# also remove rows with question_id 74-77
temp = range(74, 78)
for q in temp:
    answers_complete = answers_complete[answers_complete.question_id != q]

# first remove unnecessary columns
answers_complete = answers_complete.drop(columns = ['score', 'user_scan_id'])
answers_long = answers_complete[['user_id', 'question_id', 'value']]

# get only questions 55-73
categorical_q = range(55, 74)
answers_long = answers_long[answers_long['question_id'].isin(categorical_q)]

# include only users that have completed everything
answers_long = answers_long[answers_long['user_id'].isin(unique_user_ids)]
answers_long.head()

array([55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77])

,user_id,question_id,value
628954,1,55,9.5
628956,1,56,8
628962,1,59,Sit in a car/train/plane/bus/etc.
628963,1,60,Rarely
628964,1,61,I want to learn more about sleep


In [ ]:
# save answers in long format to csv
answers_long.to_csv('answers_long.csv')

# Create dictionary of answer keys

Create a dictionary that points to separate tables for answer keys for each pre-assessment question.

In [7]:
# subset answer key
answer_key = answer_key[answer_key.question_id.isin(categorical_q)]
answer_key = answer_key[['question_id', 'value']]
answer_key.shape

# initialize answer key dictionary
answer_key_dict = {}

# group answer key by question_id
gp = answer_key.groupby('question_id')
for grp in gp.groups:
    answer_key_dict[str(grp)] = answer_key.loc[gp.groups[grp]]

answer_key_dict['58'] # ex: small df for question_id 58

(64, 2)

,question_id,value
146,58,Woman
147,58,Man
148,58,Other


# Reshape user answers from long to wide format

Currently, the data are in long format where each row is a question and its answer. Need to convert this to wide format where each row is a user, and each column is an answer for that particular question.

In [9]:
# column names in users table
colnames = ['user_id', 'sleep_wkdy', 'sleep_wknd', 'birth',
           'Q1_woman', 'Q1_man', 'Q1_other',
           'Q2_meeting', 'Q2_movie', 'Q2_waiting', 'Q2_car', 'Q2_other', 'Q2_never',
           'Q3_5to7', 'Q3_3to4', 'Q3_1to2', 'Q3_rarely',
           'Q4_quantity', 'Q4_quality', 'Q4_learn', 'Q4_other', 'Q4_idk',
           'Q5_falling', 'Q5_staying', 'Q5_wakingearly', 'Q5_wakingtired', 'Q5_none',
           'Q6_5to7', 'Q6_3to4', 'Q6_1to2', 'Q6_rarely',
           'Q7_yes', 'Q7_no',
           'Q8_yes', 'Q8_no',
           'Q9_once', 'Q9_twice', 'Q9_three', 'Q9_overthree',
           'Q10_verycold', 'Q10_cool', 'Q10_littlewarm', 'Q10_verywarm',
           'Q11_superquiet', 'Q11_prettyquiet', 'Q11_littlenoisy', 'Q11_verynoisy',
           'Q12_completelydark', 'Q12_prettydark', 'Q12_littlebright', 'Q12_verybright',
           'Q13_yes', 'Q13_no',
           'Q14_on', 'Q14_vibrate', 'Q14_donotdisturb', 'Q14_airplane', 'Q14_off',
           'Q15_yes', 'Q15_no',
           'Q16_never', 'Q16_rarely', 'Q16_sometimes', 'Q16_often', 'Q16_veryoften']

# unique user ids
unique_user_ids = answers_wide['user_id'].unique()

users_wide = pd.DataFrame(index = unique_user_ids, columns = colnames)

# fill in user_id
users_wide['user_id'] = unique_user_ids

# fill in hours of sleep weekday
temp = answers_wide[answers_wide['question_id'] == 55][['user_id', 'question_id', 'value']]
temp_agg = temp.groupby(['user_id', 'question_id']).agg(lambda x:x.value_counts().index[0])
users_wide['sleep_wkdy'] = np.asarray(temp_agg)

# fill in hours of sleep weekend
temp = answers_wide[answers_wide['question_id'] == 56][['user_id', 'question_id', 'value']]
temp_agg = temp.groupby(['user_id', 'question_id']).agg(lambda x:x.value_counts().index[0])
users_wide['sleep_wknd'] = np.asarray(temp_agg)

# fill in birth year
temp = answers_wide[answers_wide['question_id'] == 57][['user_id', 'question_id', 'value']]
temp = temp[~(temp.value.str.len() != 4)]
temp.value = temp.value.astype(float) # convert to float
temp = temp[temp.value > 1000]
temp.value = temp.value.astype(int) # convert to int
temp = temp.drop_duplicates(subset='user_id', keep="last")
# join on user_id
users_wide = pd.merge(temp, users_wide, on='user_id', how='inner')
users_wide = users_wide.drop(columns=['question_id', 'birth'])
users_wide = users_wide.rename(columns = {'value':'age'})
# subtract birth year from current year (2018)
users_wide['age'] = 2018 - users_wide['age']


In [ ]:
# only use users that are in users_wide in answers_wide
answers_wide = answers_wide[answers_wide['user_id'].isin(users_wide.user_id)]
# only use questions that are categorical questions
categorical_q = range(58, 74)
answers_wide = answers_wide[answers_wide['question_id'].isin(categorical_q)]

# Populate users' answers wide dataframe

In [ ]:
users_wide = users_wide.set_index('user_id')

In [ ]:
# group users by user_id

gp = answers_wide.groupby('user_id')

# get number of possible answers for each multiple choice question
num_ans = answer_key[answer_key['question_id'].isin(answers_wide.question_id.unique())]
num_ans = np.array(num_ans['question_id'].value_counts().sort_index())

# get column names
cols = list(users_wide)

f = FloatProgress(min=0, max=len(gp.groups)) # instantiate progress bar
display(f) # display the bar

start = time.time()
for grp in gp.groups:
    tmp = answers_wide.loc[gp.groups[grp]]
    tmp['ans_index'] = 0
    
    for i in range(len(tmp)): # loops through current user answers
        # current question_id
        tmp_qid = tmp.iloc[i, 1]
        # current answer (value)
        tmp_val = tmp.iloc[i, 2]
        # get question df for current question
        tmp_q_df = answer_key_dict[str(tmp_qid)]
        # how many possible answers in current question
        tmp_total = len(tmp_q_df)
        # index of location for this question
        tmp.iloc[i, 3] = list(tmp_q_df.value).index(tmp_val)
        
        # populate users_wide
        if tmp['question_id'].iloc[i] in categorical_q:
            tmp_qid = tmp['question_id'].iloc[i]
            # index of start of this qid in users_wide
            tmp_start = 3 + sum(num_ans[:tmp_qid - 58])
            # fill in with a 1
            users_wide.loc[grp, cols[tmp_start + tmp.iloc[i, 3]]] = 1
    
    f.value += 1 # signal to increment the progress bar
    time.sleep(.1)
        
end = time.time()

print(end - start)


Create age group column by binning users' age. Four groups: 1-18, 19-30, 31-40, 41+.

In [ ]:
# bin age

low = 18
med = 30
high = 40

def binning(x, cut_points, labels=None):
    # min and max of x
    minval = x.min()
    maxval = x.max()
    
    # make list with min, max, and cut_points
    breaks = [minval] + cut_points + [maxval]
    
    # if no labels given, use default
    if not labels:
        labels = range(len(cut_points) + 1)
    
    # bin using pandas cut function
    x_bins = pd.cut(x, bins=breaks, labels = labels, include_lowest = True)
    return x_bins

cut_points = [low, med, high]
labels = ['1-18', '19-30', '31-40', '41+']
users_wide['age_group'] = binning(users_wide['age'], cut_points, labels)


In [ ]:
# fill NaN with 0 and save dataframe to csv
users_wide = users_wide.fillna(0)
users_wide.to_csv('users_wide.csv')

# Clean sleep tracker data

In [13]:
# read in data
sleep = pd.read_csv("/Users/Maime/Dropbox/Me/Insight/Shleep/user_data/sleep_tracker.csv")

### Datetime feature engineering

In [14]:
# convert to datatime
sleep['date'] = pd.to_datetime(sleep[['created_at']].stack()).unstack()
# get date only
sleep['date_only'] = sleep['date'].dt.date

sleep = sleep.drop(columns = ['id', 'created_at', 'updated_at', 'date'])
sleep.head()

,user_id,hours,rating,date_only
0,7,7.5,ok,2017-08-04
1,12,6.5,ok,2017-08-04
2,20,8.0,ok,2017-08-04
3,8,8.0,great,2017-08-04
4,24,8.0,great,2017-08-04


In [ ]:
# check for duplicates
len(sleep)
sleep = sleep._duplicates()
len(sleep)

# set index using user_id
sleep = sleep.set_index('user_id')
sleep['user_id'] = sleep.index.values

### Create new features 'days in app' and 'number of sleep logs'

In [ ]:
# create new column 'days_in_app' and 'num_logs' in sleep
sleep['first_log'] = 0
sleep['num_logs'] = 0

# initialize lists
days_in_app = []
mean_hours = []
mode_ratings = []

gp = sleep.groupby('user_id')

f = FloatProgress(min=0, max=len(gp.groups)) # instantiate progress bar
display(f) # display the bar

start = time.time()
for user in gp.groups:
    # current user's sleep tracker data
    tmp = sleep.loc[[user]]
    # first day current user logged sleep
    tmp_start = tmp.iloc[0, 2]
    sleep.loc[user, 'first_log'] = tmp_start
    # how many logs at current day
    sleep.loc[user, 'num_logs'] = np.array(range(1, len(tmp) + 1))
    # last sleep log entry
    days_in_app.append(tmp.iloc[-1, 5].days)
    # mean hours
    mean_hours.append(np.mean(tmp['hours']))
    # mode rating
    mode_ratings.append(tmp['rating'].mode())
        
    f.value += 1 # signal to increment the progress bar
    time.sleep(.1)
        
end = time.time()

print(end - start) # took about 45 min to run

In [ ]:
# subtract first log date from date stamp to get days since
sleep['days_since'] = sleep['date_only'] - sleep['first_log'] + 1

# convert days_since to int
sleep['days_since'] = sleep['days_since'].dt.days

# order sleep dataframe by user_id and date
sleep = sleep.sort_values(['user_id', 'date_only'])

In [ ]:
# create new column in sleep that gives day of week 
sleep['day_of_week'] = pd.to_datetime(sleep['date_only'])
sleep['day_of_week'] = sleep['day_of_week'].dt.weekday_name

# create new column in sleep that is 1 if weekday, 0 if weekend
sleep['weekday'] = sleep['day_of_week'].isin(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'])
sleep['weekday'] = sleep['weekday'].astype(int)

### Remove users who only tried the sleep tracker once, and that we do not have demographic data for. 

We only want to include in the linear mixed-effects the users who used the tracker more than once, and we have gender and age for. This is because if they only use the sleep tracker once, there is no information about the trend of their hours of sleep based on their sleep tracker usage.

In [ ]:
# drop rows where hours is missing
df = sleep
sleep = sleep.na(subset=['hours'])

# include users with pre-assessment data AND sleep tracker data
users = np.unique(df.index.values)
shared_users = np.intersect1d(users_wide.index.values, users) 
df = df.loc[shared_users]

# get age, sleep_wkdy, sleep_wknd from pre-assessment
age = pd.DataFrame(users_wide.loc[shared_users, 'age'])
age['user_id'] = age.index.values
sleep_wkdy = pd.DataFrame(users_wide.loc[shared_users, 'sleep_wkdy'])
sleep_wkdy['user_id'] = sleep_wkdy.index.values
sleep_wknd = pd.DataFrame(users_wide.loc[shared_users, 'sleep_wknd'])
sleep_wknd['user_id'] = sleep_wknd.index.values

df_lme = df.merge(age, how = 'left', on='user_id')
df_lme = df_lme.merge(sleep_wkdy, how = 'left', on='user_id')
df_lme = df_lme.merge(sleep_wknd, how = 'left', on='user_id')


Add days since last entry column.

In [ ]:
# add days since last entry column to sleep log
df_lme['days_since_last'] = 0

gp = df_lme.groupby('user_id')

f = FloatProgress(min=0, max=len(gp.groups)) # instantiate progress bar
display(f) # display the bar

start = time.time()
for user in gp.groups:
    # current user's df
    tmp = df_lme.loc[[user]]
    # days since last sleep log
    days_since_last = np.ediff1d(tmp['days_since'], to_begin=0)
    df_lme.loc[user, 'days_since_last'] = days_since_last

    f.value += 1 # signal to increment the progress bar
    time.sleep(.1)
        
end = time.time()


print(end - start)


In [ ]:
# save sleep log to csv
df_lme.to_csv('lme_sleep.csv')

In [ ]:
# remove the users who only tried sleep tracker once
df_lme_nodrop = df_lme[df_lme['days_since'] != 1]
df_lme_nodrop[df_lme_nodrop['days_since_last'] == 0] = 1

# save sleep log without users who dropped out to csv
df_lme_nodrop.to_csv('lme_nodrop.csv')

# Reshape sleep tracker data from long to wide format

For the Linear mixed-effects model to measure association between sleep tracker usage and more sleep, the data needs to be in long format. <br>
For the engagement prediction model, the data needs to be in wide format. 

In [ ]:
# initialize wide dataframe
unique_user_ids = sleep['user_id'].unique()
colnames = ['user_id', 'log_counts','days_in_app', 'mean_hours', 'mode_ratings']
sleep_wide = pd.DataFrame(index = unique_user_ids, columns = colnames)

# fill in user_id
sleep_wide['user_id'] = unique_user_ids

# fixing mode_ratings from a series of objects to a list of strings
mode_ratings2 = []
for i in mode_ratings:
    try:
        mode_ratings2.append(i[0])
    except:
        mode_ratings2.append('None')

# fill in log_counts, days_in_app, mean_hours, mode_ratings
sleep_wide['log_counts'] = log_counts
sleep_wide['days_in_app'] = days_in_app
sleep_wide['mean_hours'] = mean_hours
sleep_wide['mode_ratings'] = mode_ratings2

# add 1 to days_in_app
sleep_wide['days_in_app'] = sleep_wide['days_in_app'] + 1

# create new column of average number of logs per day
sleep_wide['logs_per_day'] = sleep_wide['log_counts']/sleep_wide['days_in_app']

# remove the few rows where days_in_app is still 0
sleep_wide = sleep_wide[sleep_wide.days_in_app != 0]

### Add users account data to sleep tracker data in wide format

In [ ]:
# only use columns that we want
users_anon = users_anon[['id', 'created_at', 'timezone', 'show_notification_suggestion', 'GDPR_accepted']]


In [ ]:
# extract date time from created_at
users_anon['create_date'] = pd.to_datetime(users_anon[['created_at']].stack()).unstack()
users_anon['create_date'] = users_anon['create_date'].dt.date

# delete original created_at column
users_anon = users_anon.drop(columns = ['created_at'])

In [ ]:
# add a new column to sleep_wide with number of days we have data for, per user (current date - create_date)
mostrecent_date = max(sleep['date_only'])
sleep_wide['account_age'] = (pd.to_datetime(mostrecent_date) - pd.to_datetime(sleep_wide['create_date'])).dt.days

In [ ]:
# join with sleep_wide on user_id and id
sleep_wide = sleep_wide.merge(users_anon, left_on = 'user_id', right_on = 'id', how = 'inner')

# reset the index and  'id'
sleep_wide = sleep_wide.set_index('user_id')
sleep_wide['user_id'] = sleep_wide.index.values
sleep_wide = sleep_wide.(columns = ['id'])

In [ ]:
# save sleep tracker dataframe in wide format to csv
sleep_wide.to_csv('sleep_tracker_wide.csv')